# INTELIGENCIA ARTIFICIAL (INF371)¶
Dr. Edwin Villanueva (evillatal@gmail.com)

## Implementando una red neuronal en Keras para predecir el riesgo de diabetes

Esta es una implementacion de una red neuronal para predecir el riego de diabetes en mujeres. Adaptado de https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/  

Es necesario tener instalado tensorflow y keras

### Cargar los datos para entrenamiento y validacion

En este ejemplo se va trabajar con el dataset  "Pima Indians", el cual es un conocido dataset de benchmark (https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv).   El dataset tiene 768 registros clinicos de pacientes mujeres, cada registro con 8 atributos numericos y un atributo categorico (clase) que indica si la paciente tuvo diabetes despues de 5 años de haberse colectado los atributos numericos. Los atributos numericods son:
   1. Number of times pregnant
   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   3. Diastolic blood pressure (mm Hg)
   4. Triceps skin fold thickness (mm)
   5. 2-Hour serum insulin (mu U/ml)
   6. Body mass index (weight in kg/(height in m)^2)
   7. Diabetes pedigree function
   8. Age (years)
   
El dataset original consta de 768 registros, el cual se ha dividido en 2 datasets para la presente experimentacion: pima-indians-diabetes_trn.csv con 728 registros para entrenamiento de la red, y pima-indians-diabetes_tst.csv con 40 registros para evaluacion de la red

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
numpy.random.seed(7)   # fija la semilla de numeros aleatorios para reproducibilidad

# Carga data de entrenamiento y coloca los atributos en X_train y la clase en Y_train
dataset_trn = numpy.loadtxt("pima-indians-diabetes_trn.csv", delimiter=",")
X_train = dataset_trn[:,0:8]
Y_train = dataset_trn[:,8]

# Carga data de test y coloca los atributos en X_test y la clase en Y_test
dataset_tst = numpy.loadtxt("pima-indians-diabetes_tst.csv", delimiter=",")
X_test = dataset_tst[:,0:8]
Y_test = dataset_tst[:,8]


### Define el modelo de red neuronal

La red que se define abajo es una red Multilayer Perceptron de 3 capas de neuronas (2 ocultas y una de salida). La primera capa tiene 12 neuronas, cada neurona recibe información de los 8 atributos de entrada (parametro imput_dim) y funcion de activacion RELU. La segunda capa tiene 8 neuronas y funcion de activacion RELU. La capa de salida es compuesta por una unica neurona con  funcion de activacion SIGMOIDE. El valor que entrega esta neurona es el  estimado numerico de la clase (entre 0 y 1)

In [ ]:
# create model
model = Sequential() # Informa que las capas que se van agregar son secuenciales
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compila el modelo de red neuronal

La compilacion del modelo busca la mejor forma de representar el modelo para ejecución eficiente en el hardware disponible (CPU o GPU). Para ello usa librerias numericas eficientes (el "backend"), tales como TensorFlow o Theano. En esta etapa se debe especificar algunas propiedades o parametros que se usaran en el entrenamiento, tales como la funcion de perdida (parametro "loss"), el metodo de optimizacion de los pesos (parametro "optimizer") y la metrica de desempeño que se colectara en el entrenamiento y prueba (parametro "metrics"). En el caso particular del presente ejemplo se usa la funcion de perdida 'binary_crossentropy' que es adecuada para problemas de clasificacion de 2 clases; metodo de optimizacion 'adam', que es una implementacion estocastica y eficiente del algoritmo gradiente descendente (http://arxiv.org/abs/1412.6980); La metrica 'accuracy' es tambien una medida estandard para medir el desempeño de clasificadores.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrena el modelo de red neuronal

Aqui se entrena el modelo con la funcion fit(), usando el optimizador y funcion de perdida especificado en la compilacion. La Funcion fit() recibe: los atributos de entrada (X_train), el vector de salida deseado (Y_train), el numero de epocas (parametro 'epochs') y el parametro 'batch_size' (numero de instancias que se le presenta a la red antes de que haga un ajuste a los pesos). La ejecucion de fit() sucede en CPU o GPU, de acuerdo a lo que se haya especificado 

In [ ]:
model.fit(X_train, Y_train, epochs=150, batch_size=10)

### Evalua el modelo de red neuronal

Una vez entrenado el modelo se puede evaluar que tan bien el modelo generaliza en instancias no observadas. Para ello se evalua el modelo en los datos de pueba llamando al metodo .evaluate() del modelo ajustado. Este metodo recibe los atributos de las instancias de prueba (X_test) y tambien las salidas que deben salir (Y_test). Internamente se hace la prediccion de la clase en todas las instancias y se compara con las salidas correctas, obteniendo asi la medida de desempeño 

In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n Desempeño del modelo en datos de prueba, %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))